In [1]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 56.5 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import math

from tabulate import tabulate
from tqdm import trange
import random
from torchmetrics.classification import Recall, Accuracy, AUROC, Precision

In [3]:
FILL_IN = "FILL_IN"

In [4]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2025-11-29 05:03:17--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip’

smsspamcollection.z     [          <=>       ] 198.65K  66.1KB/s    in 3.0s    

2025-11-29 05:03:21 (66.1 KB/s) - ‘smsspamcollection.zip’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [5]:
!unzip -o smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [6]:
!head -10 SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [7]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2025-11-29 05:03:28--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.1’

smsspamcollection.z     [          <=>       ] 198.65K  66.9KB/s    in 3.0s    

2025-11-29 05:03:31 (66.9 KB/s) - ‘smsspamcollection.zip.1’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [8]:
file_path = 'SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index = [])
with open(file_path) as f:
    for line in f.readlines():
        split = line.split('\t')
        df = pd.concat([
                df,
                pd.DataFrame.from_dict({
                    'label': [1 if split[0] == 'spam' else 0],
                    'text': [split[1]]
                })
            ],
            ignore_index=True
        )
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
text = df.text.values
labels = df.label.values

In [10]:
# Get the bert-base-uncased tokenizer and set lower case to True
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
def print_rand_sentence():
    '''Displays the tokens and respective IDs of a random text sample'''
    index = random.randint(0, len(text)-1)
    table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

╒══════════╤═════════════╕
│ Tokens   │   Token IDs │
╞══════════╪═════════════╡
│ my       │        2026 │
├──────────┼─────────────┤
│ uncles   │       27328 │
├──────────┼─────────────┤
│ in       │        1999 │
├──────────┼─────────────┤
│ atlanta  │        5865 │
├──────────┼─────────────┤
│ .        │        1012 │
├──────────┼─────────────┤
│ wish     │        4299 │
├──────────┼─────────────┤
│ you      │        2017 │
├──────────┼─────────────┤
│ guys     │        4364 │
├──────────┼─────────────┤
│ a        │        1037 │
├──────────┼─────────────┤
│ great    │        2307 │
├──────────┼─────────────┤
│ semester │       13609 │
├──────────┼─────────────┤
│ .        │        1012 │
╘══════════╧═════════════╛


In [12]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  encoding = tokenizer.encode_plus(
      input_text,
      add_special_tokens = True,
      max_length = 32,
      padding = 'max_length',
      truncation = True,
      return_attention_mask = True,
      return_tensors = 'pt'
  )

  return encoding


for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)


In [13]:
def print_rand_sentence_encoding():
    '''Displays tokens, token IDs and attention mask of a random text sample'''
    index = random.randint(0, len(text) - 1)
    tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
    token_ids = [i.numpy() for i in token_id[index]]
    attention = [i.numpy() for i in attention_masks[index]]
    table = np.array([tokens, token_ids, attention]).T
    print(
        tabulate(
            table,
            headers = ['Tokens', 'Token IDs', 'Attention Mask'],
            tablefmt = 'fancy_grid')
    )

print_rand_sentence_encoding()

╒══════════╤═════════════╤══════════════════╕
│ Tokens   │   Token IDs │   Attention Mask │
╞══════════╪═════════════╪══════════════════╡
│ [CLS]    │         101 │                1 │
├──────────┼─────────────┼──────────────────┤
│ yes      │        2748 │                1 │
├──────────┼─────────────┼──────────────────┤
│ there    │        2045 │                1 │
├──────────┼─────────────┼──────────────────┤
│ were     │        2020 │                1 │
├──────────┼─────────────┼──────────────────┤
│ many     │        2116 │                1 │
├──────────┼─────────────┼──────────────────┤
│ sweets   │       26844 │                1 │
├──────────┼─────────────┼──────────────────┤
│ [SEP]    │         102 │                1 │
├──────────┼─────────────┼──────────────────┤
│ [PAD]    │           0 │                0 │
├──────────┼─────────────┼──────────────────┤
│ [PAD]    │           0 │                0 │
├──────────┼─────────────┼──────────────────┤
│ [PAD]    │           0 │        

In [21]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    range(len(token_id)),
    test_size = val_ratio,
    stratify=labels,
    random_state = 42
)

# Train and validation sets
train_set = TensorDataset(
    token_id[train_idx],
    attention_masks[train_idx],
    labels[train_idx]
)

val_set = TensorDataset(
    token_id[val_idx],
    attention_masks[val_idx],
    labels[val_idx]
)

# Prepare DataLoader
train_dataloader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True
)

validation_dataloader = DataLoader(
    val_set,
    batch_size = batch_size
)

print(f"{len(train_set)} {len(val_set)}")

4459 1115


Define the LoRA specific layers.

In [115]:
import torch.nn as nn
import torch.nn.functional as F

# Define a LoRA Layer which has A, B and alpha parameters
class LoRALayer(torch.nn.Module):
  def __init__(self, in_dim, out_dim, rank, alpha):
    super().__init__()
    # Initialize A to be a parameter matrix of dimension in_dim by rank
    self.A = nn.Parameter(torch.randn(in_dim, rank))
    # Initialize all the elements of A via kaiming_uniform with a equal to sqrt(5) ?
    nn.init.kaiming_uniform_(self.A, a=0.05)
    # Initialize B to be a zero parameter matrix of the appropriate dimensions
    self.B = nn.Parameter(torch.zeros(rank, out_dim))
    self.A_before = nn.Parameter(torch.randn(in_dim, rank), requires_grad=False)
    self.B_before = nn.Parameter(torch.zeros(rank, out_dim), requires_grad=False)
    self.alpha = alpha

  def forward(self, x):
    # Pass x through the LoRA layer and return the new x
    # print(f" delta_A {(self.A.data - self.A_before).abs().mean()}")
    # print(f" delta_B {(self.B.data - self.B_before).abs().mean()}")

    self.A_before.data.copy_(self.A.data)
    self.B_before.data.copy_(self.B.data)
    x = self.alpha * torch.matmul(torch.matmul(x, self.A), self.B)
    return x

# Define a class LoRALinear which has a linear layer and a LoRA layer on top
class LoRALinear(torch.nn.Module):
  def __init__(self, linear, rank, alpha):
    super().__init__()
    self.linear = linear
    self.lora = LoRALayer(linear.in_features, linear.out_features, rank, alpha)

  def forward(self, x):
    # Pass x through the linear layer and also the lora layer
    return self.linear(x) + self.lora(x)

def lora_linear_replace(model, rank, alpha):
  # Use model.named_children to go through all layers and if the layer is Linear replace that layer with LoRALinear
  for name, module in model.named_children():
    # If the module is linear, replace the module in the model with a LoRA layer
    if isinstance(module, nn.Linear):
      # Replace the Linear layer with LoRALinear
      setattr(model, name, LoRALinear(module, rank, alpha))
    else:
      # Alterntively, recursively apply the same function to child modules so that other Linear layers get replaced
      lora_linear_replace(module, rank, alpha)

### Load specific versions of the model

In [155]:
from os import truncate

# Load the BertForSequenceClassification model
# Do not ouput the attentions and all hidden states
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', output_attentions=False, output_hidden_states=False)


# Turn off all gradients of the model to start
for param in model.parameters():
    param.requires_grad = False

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [156]:


# for name, parameters in model.named_parameters():
#   print(f" name {name} parameter {parameters.numel()}")


# Set to True if LORA is used; if False, fine_tune flag will be used to decide if you fine tune the entire model or just parts
use_lora  = False
# If this is False, turn off gradients
fine_tune = False
# Set total_parameters to 0; this will count the number of parameters in each case
total_parameters = 0

if use_lora:
  # Use the lora_linear to attach a LoRA layer to each linear later of the original BERT model
  lora_linear_replace(model, 8, 16)
else:
  # If fine_tune is off, turn off gradients for all layers other than classifier
  if not fine_tune:
    # Turn off all gradients; count just the 'classifier' layer which is the only one that has gradients
    for name, param in model.named_parameters():
      if 'classifier' in name:
        param.requires_grad = True
  else:
    # Turn off all gradients except for classifier
    for name, param in model.named_parameters():
      param.requires_grad = True

# Get the total number of parameters with gradients
total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)

if use_lora:
  assert(total_parameters == 1345552)
else:
  if fine_tune:
    assert(total_parameters == 109483778)
  else:
    assert(total_parameters == 1538)


In [157]:
import platform

# Pick the system you have and select GPU if you can
if platform.system() == 'Darwin':
    device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
elif platform.system() == 'Linux':
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
else:
    device = torch.device('cpu')
print(device)


cuda


In [158]:
model = model.to(device)

# Recommended number of epochs: See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2


In [159]:
# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5
# See: https://arxiv.org/pdf/1810.04805.pdf
lr = 1e-5
if not use_lora and not fine_tune:
  lr = 3e-3
  epochs = 5
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = lr,
    eps = 1e-08
)

In [160]:
import torchmetrics
# Use torchmetrics to set up accuracy, recall, precision, and auroc
accuracy = torchmetrics.Accuracy(task='binary').to(device)
recall = torchmetrics.Recall(task='binary').to(device)
precision = torchmetrics.Precision(task='binary').to(device)
auroc = torchmetrics.AUROC(task='binary').to(device)

### Set the model to the right device

### Train the model

In [161]:
# Main training / validation loop
import tqdm
for _ in tqdm.tqdm(trange(epochs, desc = 'Epoch')):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        # batch = FILL_IN
        # Unpack the batch
        # print(f"{batch[0]} \n {batch[1]} \n {batch[2]} \n")
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Set the gradients to zero
        optimizer.zero_grad()

        # Forward pass
        train_output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        # print(train_output)
        # print(f"output: {train_output.logits[0:4]} loss: {train_output.loss}")

        # Backward pass
        loss = train_output.loss
        loss.backward()

        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1

        if nb_tr_steps % 20 == 0:
          print(f"loss: {tr_loss / nb_tr_steps}")

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        # batch = FILL_IN
        input_ids, attention_mask, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        with torch.no_grad():
          # Forward pass
            eval_output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        # Calculate validation metrics
        # labels = FILL_IN
        predicted_labels = torch.argmax(eval_output.logits, dim=1)

        val_accuracy.append(accuracy(predicted_labels, labels))
        val_recall.append(recall(predicted_labels, labels))
        val_precision.append(precision(predicted_labels, labels))
        val_auroc.append(auroc(predicted_labels, labels))

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

  0%|          | 0/5 [00:00<?, ?it/s]

loss: 0.35874453466385603
loss: 0.3474930782802403
loss: 0.32370185138036806
loss: 0.3066373543348163
loss: 0.2992534554377198
loss: 0.29756096728766956
loss: 0.29034365572567494
loss: 0.27820998991373924
loss: 0.2738137721394499
loss: 0.2656574709853157
loss: 0.26511195790953934
loss: 0.25802836591610684
loss: 0.25543501110245975


/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)

Epoch:  20%|██        | 1/5 [00:04<00:18,  4.73s/it]


	 - Train loss: 0.2508
	 - Validation Accuracy: 0.9607
	 - Validation Precision: 0.7919
	 - Validation Recall: 0.6945
	 - Validation AUROC: 0.8000

loss: 0.18766259923577308
loss: 0.1632577775977552
loss: 0.22430281465252241
loss: 0.22233124787453562
loss: 0.20672525437548755
loss: 0.200771948400264
loss: 0.1958204216589885
loss: 0.18827040862524883
loss: 0.18056932850223448
loss: 0.17601567230653017
loss: 0.17518047480420632
loss: 0.17029025366840264
loss: 0.16800302031378334



Epoch:  40%|████      | 2/5 [00:09<00:14,  4.83s/it]

...


	 - Train loss: 0.1314
	 - Validation Accuracy: 0.9589
	 - Validation Precision: 0.7450
	 - Validation Recall: 0.8210
	 - Validation AUROC: 0.8521

loss: 0.15811661807820201
loss: 0.1309244483942166
loss: 0.14470720196453232
loss: 0.14448342515388504
loss: 0.13348990190774201
loss: 0.1360435798453788
loss: 0.13678980888798833
loss: 0.1370983522385359
loss: 0.14370986899981897
loss: 0.14748445130884646
loss: 0.14381026224317875
loss: 0.14623941594424347
loss: 0.14223252112189164



Epoch:  80%|████████  | 4/5 [00:19<00:05,  5.07s/it]


	 - Train loss: 0.1387
	 - Validation Accuracy: 0.9714
	 - Validation Precision: 0.8098
	 - Validation Recall: 0.7848
	 - Validation AUROC: 0.8430

loss: 0.10340311429463327
loss: 0.1108990875305608
loss: 0.10541478019828597
loss: 0.11218660711310804
loss: 0.11579915015026927
loss: 0.1210044125871112
loss: 0.11883128817592348
loss: 0.11496998228831216
loss: 0.12138281246750719
loss: 0.11933608420658856
loss: 0.11838088289157234
loss: 0.11568750684770444
loss: 0.11396849884006839



100%|██████████| 5/5 [00:25<00:00,  5.07s/it]


	 - Train loss: 0.1130
	 - Validation Accuracy: 0.9732
	 - Validation Precision: 0.8174
	 - Validation Recall: 0.7498
	 - Validation AUROC: 0.8287



### Test on a specific sentence, see the outcome

In [154]:
new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Spam' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Ham'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Predicted Class:  Spam


### Questions

Question 1: Run the above by fine tuning bert and the classfier head and by not doing this (using BERT as a feature encoder). What is the gap between this?

- LoRA
Epoch:   0%|          | 0/2 [00:00<?, ?it/s]
  0%|          | 0/2 [00:00<?, ?it/s]loss: 0.3316247998736799
loss: 0.22495780410245061
loss: 0.19098539882494758
loss: 0.16019987532326924
loss: 0.13725689193452126
loss: 0.11884758243286342
loss: 0.10939879653441105
loss: 0.10450777633695907
loss: 0.10035888845927224
loss: 0.09592804029489344
loss: 0.09168061471394719
loss: 0.08794965063249643
loss: 0.08347696445746876
/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)

Epoch:  50%|█████     | 1/2 [00:17<00:17, 17.38s/it]
	 - Train loss: 0.0818
	 - Validation Accuracy: 0.9857
	 - Validation Precision: 0.8560
	 - Validation Recall: 0.8155
	 - Validation AUROC: 0.8627

loss: 0.02956431172788143
loss: 0.02438367926515639
loss: 0.024925138312391938
loss: 0.021259430096688448
loss: 0.02336106476839632
loss: 0.02362940770787342
loss: 0.02158855397356092
loss: 0.024982489646481552
loss: 0.023953555756775637
loss: 0.02501407723648299
loss: 0.025323687590174483
loss: 0.027551542510688403
loss: 0.02835389203979643

Epoch: 100%|██████████| 2/2 [00:34<00:00, 17.43s/it]
100%|██████████| 2/2 [00:34<00:00, 17.43s/it]
	 - Train loss: 0.0277
	 - Validation Accuracy: 0.9866
	 - Validation Precision: 0.8447
	 - Validation Recall: 0.8131
	 - Validation AUROC: 0.8613

- fine tune all

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]
  0%|          | 0/2 [00:00<?, ?it/s]loss: 0.548489311337471
loss: 0.4259004842489958
loss: 0.361080460002025
loss: 0.3116921674925834
loss: 0.27456370983272793
loss: 0.24498644994261365
loss: 0.21925580415076443
loss: 0.20055850737844594
loss: 0.1836392905459636
loss: 0.1718765657255426
loss: 0.1613125826528465
loss: 0.14959105977904982
loss: 0.1410685462698054
/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)

Epoch:  50%|█████     | 1/2 [00:21<00:21, 21.35s/it]
	 - Train loss: 0.1352
	 - Validation Accuracy: 0.9866
	 - Validation Precision: 0.8845
	 - Validation Recall: 0.8483
	 - Validation AUROC: 0.8791

loss: 0.030276984348893164
loss: 0.03264863586518914
loss: 0.0269742448348552
loss: 0.02214935360243544
loss: 0.023893477977253497
loss: 0.026835770503384992
loss: 0.027689395583833435
loss: 0.030076089248177597
loss: 0.029833797649997804
loss: 0.03061840495909564
loss: 0.028709682419007135
loss: 0.027502108573389705
loss: 0.02723947230887671

Epoch: 100%|██████████| 2/2 [00:42<00:00, 21.44s/it]
100%|██████████| 2/2 [00:42<00:00, 21.44s/it]
	 - Train loss: 0.0270
	 - Validation Accuracy: 0.9884
	 - Validation Precision: 0.8828
	 - Validation Recall: 0.8702
	 - Validation AUROC: 0.8894

- fine tune classifier

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]
  0%|          | 0/5 [00:00<?, ?it/s]loss: 0.35874453466385603
loss: 0.3474930782802403
loss: 0.32370185138036806
loss: 0.3066373543348163
loss: 0.2992534554377198
loss: 0.29756096728766956
loss: 0.29034365572567494
loss: 0.27820998991373924
loss: 0.2738137721394499
loss: 0.2656574709853157
loss: 0.26511195790953934
loss: 0.25802836591610684
loss: 0.25543501110245975
/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)

Epoch:  20%|██        | 1/5 [00:04<00:18,  4.73s/it]
	 - Train loss: 0.2508
	 - Validation Accuracy: 0.9607
	 - Validation Precision: 0.7919
	 - Validation Recall: 0.6945
	 - Validation AUROC: 0.8000

loss: 0.18766259923577308
loss: 0.1632577775977552
loss: 0.22430281465252241
loss: 0.22233124787453562
loss: 0.20672525437548755
loss: 0.200771948400264
loss: 0.1958204216589885
loss: 0.18827040862524883
loss: 0.18056932850223448
loss: 0.17601567230653017
loss: 0.17518047480420632
loss: 0.17029025366840264
loss: 0.16800302031378334

Epoch:  40%|████      | 2/5 [00:09<00:14,  4.83s/it]
	 - Train loss: 0.1670
	 - Validation Accuracy: 0.9259
	 - Validation Precision: 0.5714
	 - Validation Recall: 0.3993
	 - Validation AUROC: 0.6568

loss: 0.10671788621693849
loss: 0.11243802774697542
loss: 0.11816457097108166
loss: 0.1255414822837338
loss: 0.12705017598345875
loss: 0.1285168319164465
loss: 0.12601323892761554
loss: 0.12480269988300279
loss: 0.12673915893667273
loss: 0.12886800840497017
loss: 0.12979534811615437
loss: 0.1336635513822936
loss: 0.13184856941225007

Epoch:  60%|██████    | 3/5 [00:14<00:09,  4.94s/it]
	 - Train loss: 0.1314
	 - Validation Accuracy: 0.9589
	 - Validation Precision: 0.7450
	 - Validation Recall: 0.8210
	 - Validation AUROC: 0.8521

loss: 0.15811661807820201
loss: 0.1309244483942166
loss: 0.14470720196453232
loss: 0.14448342515388504
loss: 0.13348990190774201
loss: 0.1360435798453788
loss: 0.13678980888798833
loss: 0.1370983522385359
loss: 0.14370986899981897
loss: 0.14748445130884646
loss: 0.14381026224317875
loss: 0.14623941594424347
loss: 0.14223252112189164

Epoch:  80%|████████  | 4/5 [00:19<00:05,  5.07s/it]
	 - Train loss: 0.1387
	 - Validation Accuracy: 0.9714
	 - Validation Precision: 0.8098
	 - Validation Recall: 0.7848
	 - Validation AUROC: 0.8430

loss: 0.10340311429463327
loss: 0.1108990875305608
loss: 0.10541478019828597
loss: 0.11218660711310804
loss: 0.11579915015026927
loss: 0.1210044125871112
loss: 0.11883128817592348
loss: 0.11496998228831216
loss: 0.12138281246750719
loss: 0.11933608420658856
loss: 0.11838088289157234
loss: 0.11568750684770444
loss: 0.11396849884006839

Epoch: 100%|██████████| 5/5 [00:25<00:00,  5.07s/it]
100%|██████████| 5/5 [00:25<00:00,  5.07s/it]
	 - Train loss: 0.1130
	 - Validation Accuracy: 0.9732
	 - Validation Precision: 0.8174
	 - Validation Recall: 0.7498
	 - Validation AUROC: 0.8287


